In [1]:
# https://towardsdatascience.com/backtesting-with-python-109cab7ee737

#&apikey=33a7c94e046defe591808732c7e21281

In [15]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [18]:
#apikey=33a7c94e046defe591808732c7e21281
#  Visit https://financialmodelingprep.com/developer/docs/

In [95]:
test_stock = 'TSLA'
api_key='33a7c94e046defe591808732c7e21281'
stockprices = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/{test_stock}?serietype=line&apikey={api_key}")
stockprices = stockprices.json()['historical']

In [97]:
stockprices[0:10]

[{'date': '2020-06-26', 'close': 959.74},
 {'date': '2020-06-25', 'close': 985.98},
 {'date': '2020-06-24', 'close': 960.85},
 {'date': '2020-06-23', 'close': 1001.78},
 {'date': '2020-06-22', 'close': 994.32},
 {'date': '2020-06-19', 'close': 1000.9},
 {'date': '2020-06-18', 'close': 1003.96},
 {'date': '2020-06-17', 'close': 991.79},
 {'date': '2020-06-16', 'close': 982.13},
 {'date': '2020-06-15', 'close': 990.9}]

In [98]:
stockprices[-20:]

[{'date': '2010-07-27', 'close': 20.55},
 {'date': '2010-07-26', 'close': 20.95},
 {'date': '2010-07-23', 'close': 21.29},
 {'date': '2010-07-22', 'close': 21.0},
 {'date': '2010-07-21', 'close': 20.22},
 {'date': '2010-07-20', 'close': 20.3},
 {'date': '2010-07-19', 'close': 21.91},
 {'date': '2010-07-16', 'close': 20.64},
 {'date': '2010-07-15', 'close': 19.89},
 {'date': '2010-07-14', 'close': 19.84},
 {'date': '2010-07-13', 'close': 18.14},
 {'date': '2010-07-12', 'close': 17.05},
 {'date': '2010-07-09', 'close': 17.4},
 {'date': '2010-07-08', 'close': 17.46},
 {'date': '2010-07-07', 'close': 15.8},
 {'date': '2010-07-06', 'close': 16.11},
 {'date': '2010-07-02', 'close': 19.2},
 {'date': '2010-07-01', 'close': 21.96},
 {'date': '2010-06-30', 'close': 23.83},
 {'date': '2010-06-29', 'close': 23.89}]

- First (1), we create a new column that will contain True for all data points in the data frame where the 20 days moving average cross above the 250 days moving average.

- Of course, we are only interested in the first or second day when the crossover happens (i.e. 20 days MA goes over 250 days MA). Therefore, we locate the first or second date (rows) where the crossover happens (2).

- As well stated in this article, we will use the two-day rule ( ie we start the trade only after it is confirmed by one more day’s closing), and will keep the date as the entry point only if the 20 days MA is above 250 days MA two days in a row.

In [125]:
def stockpriceanalysis(stock):
    stockprices = requests.get(f"https://financialmodelingprep.com/api/v3/historical-price-full/{stock}?serietype=line&apikey={api_key}")
    stockprices = stockprices.json()
    #print(stockprices)
#Parse the API response and select only last 1200 days of prices
    stockprices = stockprices['historical']
    stockprices = stockprices[0 : 1200]
    stockprices.reverse()    # $$$  I had to reverse the array to get it working.
 #   print(stockprices[-20:])

    stockprices = pd.DataFrame.from_dict(stockprices)
    stockprices = stockprices.set_index('date')
    
    #20 days to represent the 22 trading days in a month
    stockprices['20d'] = stockprices['close'].rolling(20).mean()
    stockprices['250d'] = stockprices['close'].rolling(250).mean()
    
    ###STARTING BACKTESTING STRATEGY
    #Start longposiiton list
    longpositionsprofit = []

    shortpositions = []

    stockprices['buy'] =(stockprices['20d'] > stockprices['250d'])

    #We find the first True since it is first day when the line has crossed. 
    #Also, ensure that we have at least two days where sticoprices
    
    stockprices['firstbuy'] =   ((stockprices['buy'] == True) & 
                                 (stockprices['buy'].shift(2) == False)& 
                                 (stockprices['buy'].shift(1) == True))


    buyingpoints = np.where(stockprices['firstbuy'] == True)

    for item in buyingpoints[0]:
        #close price of the stock when MA crosses
        close_price = stockprices.iloc[item].close
        #Enter long position
        long = close_price*100
        sellday = item + 20
        close_price = stockprices.iloc[sellday].close
        #Sell 20 days later:
        sell = close_price*100

        #Calculate profti
        profit = sell - long

        longpositionsprofit.append(profit)
        
    #Result of Moving Average Strategy of going long
    print("Long Position Profit")
    print(f"Profit of Moving Average Strategy = ${sum(longpositionsprofit)}" )
    #Result of just holding the stock over 1200 days
  
    firstdayprice = stockprices.iloc[0].close
    lastdayprice = stockprices.iloc[-1].close

    profit = 100*(lastdayprice - firstdayprice)
    print(f"Buy and Hold Proft = ${profit}.  Start-Price = ${firstdayprice}. End-Price = ${lastdayprice}.  ")


In [126]:
stockpriceanalysis('aapl')

Long Position Profit
Profit of Moving Average Strategy = $1913.0
Buy and Hold Proft = $24023.0.  Start-Price = $113.4. End-Price = $353.63.  


In [127]:
stockpriceanalysis('MSFT')

Long Position Profit
Profit of Moving Average Strategy = $75.0
Buy and Hold Proft = $15243.0.  Start-Price = $43.9. End-Price = $196.33.  


In [128]:
stockpriceanalysis('AMZN')

Long Position Profit
Profit of Moving Average Strategy = $31290.0
Buy and Hold Proft = $215446.99999999997.  Start-Price = $538.4. End-Price = $2692.87.  


In [129]:
stockpriceanalysis('TSLA')

Long Position Profit
Profit of Moving Average Strategy = $-595.0000000000036
Buy and Hold Proft = $69880.0.  Start-Price = $260.94. End-Price = $959.74.  
